In [1]:
import os 
os.chdir('/lustre/scratch/kiviaho/prostate_spatial/')

import numpy as np
import anndata as ad
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

from  scripts.utils import get_sample_ids,spatially_aware_clustering,load_from_pickle,save_to_pickle

In [4]:
#### CLUSTER AND PLOT SAMPLES INDIVIDUALLY ###

# LOAD NORMALIZED DATA IN A DICT
normalized_adata = load_from_pickle('./data/normalized_visium_data.pickle')
weight = 0.3
res = 1.0

# This is for doing clustering on the original data & plotting the clusters to see if they match tissue features. No spotclean, just scanpy normalization
samples = get_sample_ids()
for sample_id in samples:
    normalized_adata[sample_id] = spatially_aware_clustering(normalized_adata[sample_id],proximity_weight=weight,res=res)


In [5]:


# Plot the individual sample clusters
weight = str(weight)
res = str(res)
save = True

for sample_id in samples:
    adata = normalized_adata[sample_id]

    sq.pl.spatial_scatter(adata,color=['joint_leiden_clusters'],size=1.2,figsize=(6,6),dpi=120)
    if save:
        # Save a spatial plot with individual sample clustering
        img_dir_path = './plots/normalized_spatial_clustering_weight_'+weight+'_resolution_'+res+'_FINAL'
        Path(img_dir_path).mkdir(parents=True, exist_ok=True)
        plt.savefig(img_dir_path+'/'+sample_id+'_normalized_spatial_weight_'+weight+'_resolution_'+res+'.png')
        plt.clf()
        # Save the cluster labels into a csv format for import into Loupe browser
        label_dir_path = './plots/labels_normalized_spatial_clustering_weight_'+weight+'_resolution_'+res+'_FINAL'
        Path(label_dir_path).mkdir(parents=True, exist_ok=True)
        df_to_save = pd.DataFrame({'Barcode':adata.obs.index,'Spatial cluster': ['Cluster '+ cl for cl in adata.obs.joint_leiden_clusters]})
        df_to_save.to_csv(label_dir_path+'/'+sample_id+'_clusters_proximity_weight_'+weight+'_resolution_'+res+'.csv',index=False)


/home/ak431480/.conda/envs/squidpy/lib/python3.8/site-packages/squidpy/pl/_spatial_utils.py:819: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=figsize, dpi=dpi, constrained_layout=True)


<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

...

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

<Figure size 720x720 with 0 Axes>

In [6]:
# Save the individually clustered spatial data
save_to_pickle(normalized_adata,'./data/clustered_visium_data.pickle')

## Initializing a Loupe browser pathology layer 

In [10]:
## Save a mockup pathology layer for importing to loupe browser. The categories are as follows:
pathology_classes = ['Gleason 3',
                    'Gleason 4',
                    'Gleason 4 cribriform',
                    'Gleason 5',
                    'Stroma',
                    'Inflammation',
                    'Exclude',
                    'Lumen',
                    'Benign',
                    'PIN']
# Repeat each 3 times to not have them removed when working in loupe.
pathology_classes = [item for item in pathology_classes for _ in range(3)]

label_dir_path = './plots/pathology_layers_for_cloupe'
Path(label_dir_path).mkdir(parents=True, exist_ok=True)

from itertools import zip_longest

for sample_id in samples:
    adata = normalized_adata[sample_id]

    barcodes = [element.rsplit('_', 1)[-1] for element in adata.obs.index]

    df_to_save = pd.DataFrame(zip_longest(barcodes, pathology_classes), columns=['Barcode', 'Pathology'])
    df_to_save.to_csv(label_dir_path+'/'+sample_id+'_pathology_classes.csv',index=False)